In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/CR_cells_19pcw+14pcw_AnnData_1.h5ad", "rb"))

In [3]:
adata = adata[adata.obs['Age'] == '14 pcw',:]

In [4]:
adata.obs['nuclei'] = adata.obs['nuclei'].astype(float)

Trying to set attribute `.obs` of view, copying.


In [5]:
# Number of nuclei per AOI type:
print('CR cell ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcellsB']))
print('Residual ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']))
# RELN expression per AOI type:
print('CR cell ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'CRcellsB',np.array(adata.var == 'RELN')].X))
print('Residual ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X))
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcellsB']) > np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']):
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcellsB',np.array(adata.var == 'RELN')].X) < np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X):
    raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data looks sensible.')

CR cell ROIs mean nuclei: 24.675
Residual ROIs mean nuclei: 91.0
CR cell ROIs mean RELN expression: 292.8
Residual ROIs mean RELN expression: 15.175
Frontal Cortex ROIs mean NR2F1 expression: 23.3125
Occipital Cortex ROIs mean NR2F1 expression: 23.34375
Input data looks sensible.


In [6]:
adata.var_names = adata.var['SYMBOL']

In [7]:
adata = adata[adata.obs['Pool'] == 'CRcellsB',:]

In [8]:
adata.X = adata.layers['X_corrected_naive_cpm']

In [9]:
adata.obs['Tissue'].value_counts()

Foetal brain (frontal lobe)      16
Foetal brain (occipital lobe)    16
Foetal brain (temporal lobe)      8
Name: Tissue, dtype: int64

In [10]:
adata.obs['Donor'].value_counts()

Hob5    40
Name: Donor, dtype: int64

In [11]:
adata.X = adata.layers['X_corrected_naive_cpm']
res = {}
count = 0
tissues = np.unique(adata.obs['Tissue'])
for i in range(len(tissues)):
    for j in range((i+1),len(tissues)):
        print(i)
        print(j)
        if i !=j:
            new_key = str(tissues[i]) + '_vs_' + str(tissues[j])
            adata_subset = adata[[x in (tissues[i], tissues[j]) for x in adata.obs['Tissue']],:]
            print(new_key)
            print((adata_subset.obs['Tissue'].astype(str) + ' ' + adata_subset.obs['Pool'].astype(str)
 + ' ' + adata_subset.obs['Donor'].astype(str)).value_counts())
            res[new_key] = de.test.rank_test(
                data=adata_subset,
                grouping="Tissue",
                is_logged=False
            ).summary()
            
for i in range(len(res.keys())):
    
    print(list(res.keys())[i])
    
    tissue1 = list(res.keys())[i].split('_vs_')[0]
    tissue2 = list(res.keys())[i].split('_vs_')[1]
    print(list(res.keys())[i])
    print(tissue1)
    print(tissue2)
    
    if np.sum(np.array(res[list(res.keys())[i]]['gene']) == np.array(adata.var_names)) != len(adata.var_names):
        raise ValueError()
    
    res[list(res.keys())[i]]['log2_mean_' + tissue1] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_' + tissue2] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change'] =  res[list(res.keys())[i]]['log2_mean_' + tissue1] - res[list(res.keys())[i]]['log2_mean_' + tissue2]
    
    res[list(res.keys())[i]] = res[list(res.keys())[i]].sort_values('pval')
    
res_naive = res

0
1
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (occipital lobe) CRcellsB Hob5    16
Foetal brain (frontal lobe) CRcellsB Hob5      16
dtype: int64
0
2
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe) CRcellsB Hob5     16
Foetal brain (temporal lobe) CRcellsB Hob5     8
dtype: int64
1
2
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (occipital lobe) CRcellsB Hob5    16
Foetal brain (temporal lobe) CRcellsB Hob5      8
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe)
Foetal brain (occipital lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (te

/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in log2


In [12]:
pickle.dump(res_naive, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_14pcw_pairwiseDE_naive.p", "wb" ))

In [13]:
adata.X = adata.layers['X_corrected_cc_cpm']
res = {}
count = 0
tissues = np.unique(adata.obs['Tissue'])
for i in range(len(tissues)):
    for j in range((i+1),len(tissues)):
        print(i)
        print(j)
        if i !=j:
            new_key = str(tissues[i]) + '_vs_' + str(tissues[j])
            adata_subset = adata[[x in (tissues[i], tissues[j]) for x in adata.obs['Tissue']],:]
            print(new_key)
            print((adata_subset.obs['Tissue'].astype(str) + ' ' + adata_subset.obs['Pool'].astype(str)
 + ' ' + adata_subset.obs['Donor'].astype(str)).value_counts())
            res[new_key] = de.test.rank_test(
                data=adata_subset,
                grouping="Tissue",
                is_logged=False
            ).summary()
            
for i in range(len(res.keys())):
    
    print(list(res.keys())[i])
    
    tissue1 = list(res.keys())[i].split('_vs_')[0]
    tissue2 = list(res.keys())[i].split('_vs_')[1]
    print(list(res.keys())[i])
    print(tissue1)
    print(tissue2)
    
    if np.sum(np.array(res[list(res.keys())[i]]['gene']) == np.array(adata.var_names)) != len(adata.var_names):
        raise ValueError()
    
    res[list(res.keys())[i]]['log2_mean_' + tissue1] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_' + tissue2] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change'] =  res[list(res.keys())[i]]['log2_mean_' + tissue1] - res[list(res.keys())[i]]['log2_mean_' + tissue2]
    
    res[list(res.keys())[i]] = res[list(res.keys())[i]].sort_values('pval')
    
res_cc = res

0
1
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (occipital lobe) CRcellsB Hob5    16
Foetal brain (frontal lobe) CRcellsB Hob5      16
dtype: int64
0
2
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe) CRcellsB Hob5     16
Foetal brain (temporal lobe) CRcellsB Hob5     8
dtype: int64
1
2
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (occipital lobe) CRcellsB Hob5    16
Foetal brain (temporal lobe) CRcellsB Hob5      8
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe)
Foetal brain (occipital lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (te

/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/cellpymc/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in log2


In [14]:
pickle.dump(res_cc, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_14pcw_pairwiseDE_cc.p", "wb" ))

In [35]:
allResults = []

include_sc_array = [True, False, False]
key_wta_array = ['Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)',
                'Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)',
                'Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)']
key_sc_array = ['PFC_vs_V1', None, None]

for j in range(len(key_wta_array)):
    
    print(j)

    res_sc = pickle.load(open("/nfs/team283/brainData/human_fetal/Bhaduri2021/CR_cells_pairwiseDE.p", "rb" ))
    res_cc = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_14pcw_pairwiseDE_cc.p", "rb" ))
    res_naive = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_14pcw_pairwiseDE_naive.p", "rb" ))
    
    tissue1_wta = key_wta_array[j].split('_vs_')[0]
    tissue2_wta = key_wta_array[j].split('_vs_')[1]
    
    include_sc = include_sc_array[j]
    
    if include_sc:
        tissue1_sc = key_sc_array[j].split('_vs_')[0]
        tissue2_sc = key_sc_array[j].split('_vs_')[1]

    key_wta = key_wta_array[j]
    key_sc = key_sc_array[j]

    if include_sc:
        res_sc = res_sc[key_sc]
    res_cc = res_cc[key_wta]
    res_naive = res_naive[key_wta]
    res_tidy = pd.DataFrame(index = list(range(len(res_cc['gene']))), columns = ('gene', 'qval_cc', 'qval_naive', 'qval_sc',
                                       'log2_mean_wta' + tissue1_wta, 'log2_mean_wta' + tissue2_wta,
                                       'log2_mean_sc' + tissue1_sc, 'log2_mean_sc' + tissue2_sc,                                         
                                       'log2fc_cc', 'log2fc_naive', 'log2fc_sc'))
    
    print(key_wta)
    print(key_sc)
    print(tissue1_wta)
    print(tissue2_wta)
    
    for i in range(len(res_cc['gene'])):
        res_tidy['gene'].iloc[i]  = res_cc['gene'].iloc[i]
        
        index_naive = res_naive['gene'] == res_cc['gene'].iloc[i]
        if include_sc:
            index_sc = res_sc['gene'] == res_cc['gene'].iloc[i]
        else:
            index_sc = (False, False, False)
        
        if res_naive['gene'].loc[index_naive].iloc[0] != res_cc['gene'].iloc[i]:
            print(res_naive['gene'].loc[index_naive])
            print(res_cc['gene'].iloc[i])
            raise ValueError()
        
        res_tidy['qval_cc'].iloc[i] = res_cc['qval'].iloc[i]
        res_tidy['qval_naive'].iloc[i]  = res_naive['qval'].loc[index_naive].iloc[0]
        if np.sum(index_sc):
            res_tidy['qval_sc'].iloc[i]  = res_sc['qval'].loc[index_sc].iloc[0]
        
        res_tidy['log2_mean_wta' + tissue1_wta].iloc[i] = res_cc['log2_mean_' + tissue1_wta].iloc[i]
        res_tidy['log2_mean_wta' + tissue2_wta].iloc[i] = res_cc['log2_mean_' + tissue2_wta].iloc[i]
        
        if np.sum(index_sc):
            res_tidy['log2_mean_sc' + tissue1_sc].iloc[i] = res_sc['log2_mean_' + tissue1_sc].loc[index_sc].iloc[0]
            res_tidy['log2_mean_sc' + tissue2_sc].iloc[i] = res_sc['log2_mean_' + tissue2_sc].loc[index_sc].iloc[0]
        
        res_tidy['log2fc_cc'].iloc[i]  = res_cc['log2_fold_change'].iloc[i]
        res_tidy['log2fc_naive'].iloc[i]  = res_naive['log2_fold_change'].loc[index_naive].iloc[0]
        if np.sum(index_sc):
            res_tidy['log2fc_sc'].iloc[i]  = res_sc['log2_fold_change'].loc[index_sc].iloc[0]
        
#         if res_tidy['log2fc_cc'].iloc[i] != (res_tidy['log2_mean_wta' + tissue1_wta].iloc[i] - res_tidy['log2_mean_wta' + tissue2_wta].iloc[i]):
#             raise ValueError()
            
    allResults += [res_tidy]

0
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
PFC_vs_V1
Foetal brain (frontal lobe)
Foetal brain (occipital lobe)
1
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
None
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
2
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
None
Foetal brain (occipital lobe)
Foetal brain (temporal lobe)


In [37]:
pickle.dump(allResults, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_DE_14pcw_pairwise_allResults.p", "wb" ))

In [38]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cell_DE_14pcw_pairwise_results.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in range(len(allResults)):
    allResults[i].to_excel(writer, sheet_name=key_wta_array[i].split('Foetal brain')[1].split('lobe')[0].split('(')[1]
                           + 'vs '
                           + key_wta_array[i].split('Foetal brain')[2].split('lobe')[0].split('(')[1])

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [39]:
allResults[0]

gene   qval_cc qval_naive   qval_sc  \
0           PDE4DIP  0.137316   0.252056   0.65228   
1              NSMF  0.137316   0.252056   0.65228   
2            CLDN11  0.137316   0.173266       NaN   
3            ACVR2A  0.137316   0.252056  0.944101   
4           ZCCHC24  0.137316   0.300286   0.65228   
...             ...       ...        ...       ...   
18672         GINS3       1.0        1.0       NaN   
18673         SYTL1       1.0   0.915672   0.65228   
18674         ADCY7       1.0        1.0       NaN   
18675        CLEC2A       1.0        1.0       NaN   
18676  NegProbe-WTX       NaN   0.926567       NaN   

      log2_mean_wtaFoetal brain (frontal lobe)  \
0                                     9.923353   
1                                     8.763732   
2                                     2.549191   
3                                      8.22418   
4                                     6.341083   
...                                        ...   
18672                                 3.403363   
18673                                 2.623848   
18674                                 3.140782   
18675                                 2.716229   
18676                                     -inf   

      log2_mean_wtaFoetal brain (occipital lobe) log2_mean_scPFC  \
0                                       8.811675        1.303745   
1                                       7.970868        0.267608   
2                                       6.472014             0.0   
3                                       6.537645        0.283368   
4                                       8.052313             0.0   
...                                          ...             ...   
18672                                   3.850281             0.0   
18673                                   3.018656        0.269904   
18674                                   3.258211             0.0   
18675                                   3.057886             0.0   
18676                                       -inf             NaN   

      log2_mean_scV1 log2fc_cc log2fc_naive log2fc_sc  
0           0.638549  1.111678     0.979535  0.665196  
1           0.837392  0.792864     0.637938 -0.569784  
2                0.0 -3.922824     -2.65393       0.0  
3           0.506874  1.686535     1.273989 -0.223507  
4           0.257524  -1.71123    -2.629295 -0.257524  
...              ...       ...          ...       ...  
18672            0.0 -0.446918    -0.805599       0.0  
18673            0.0 -0.394807    -0.847708  0.269904  
18674            0.0 -0.117429    -0.246002       0.0  
18675            0.0 -0.341656    -0.252661       0.0  
18676            NaN       NaN         -inf       NaN  

[18677 rows x 11 columns]

In [63]:
allResults[0].loc[[allResults[0]['qval_cc'].iloc[i] < 0.25 and np.abs(allResults[0]['log2fc_cc'].iloc[i]) > 2
 for i in range(len(allResults[0]['qval_cc']))],:]

gene   qval_cc qval_naive   qval_sc  \
2    CLDN11  0.137316   0.173266       NaN   
5      CERK  0.162481    0.25465   0.91936   
7     KCNJ9  0.162481   0.252056       NaN   
10    SARNP  0.162481   0.288422   0.65228   
14    CSRP2  0.162481   0.283788  0.905163   
17    KLHL8  0.162481   0.252056   0.92664   
19    FBLL1  0.165083   0.252056   0.65228   
20    PSMC5  0.165083   0.260273  0.754214   
26     SQLE  0.165083   0.295012  0.684673   
27   SEC16A  0.184354   0.252056   0.65228   
28  ANKRD50  0.185772   0.288422   0.65228   
34  CHORDC1  0.209595   0.292764   0.65228   
35  SUPT20H  0.217837   0.312297   0.65228   

   log2_mean_wtaFoetal brain (frontal lobe)  \
2                                  2.549191   
5                                  6.896925   
7                                  4.875624   
10                                 6.779169   
14                                 7.582761   
17                                 6.107289   
19                                 6.178734   
20                                 6.333797   
26                                 6.167015   
27                                 6.205242   
28                                 6.047958   
34                                 5.397183   
35                                 5.739372   

   log2_mean_wtaFoetal brain (occipital lobe) log2_mean_scPFC log2_mean_scV1  \
2                                    6.472014             0.0            0.0   
5                                    4.794065        0.206798       0.233323   
7                                    7.184908             0.0            0.0   
10                                   4.666394        0.635266       0.111789   
14                                   5.559118         3.29593       2.471354   
17                                   2.870342        0.287481         0.3269   
19                                   2.964582             0.0       0.244947   
20                                   4.193357        1.249715       1.485037   
26                                   3.947591        1.810892       2.004948   
27                                   2.527776        0.075575            0.0   
28                                   2.947846        0.392555            0.0   
34                                   2.556659        0.779248       0.182558   
35                                   2.046005        0.450814       0.154479   

   log2fc_cc log2fc_naive log2fc_sc  
2  -3.922824     -2.65393       0.0  
5    2.10286     1.618494 -0.026525  
7  -2.309284    -1.314236       0.0  
10  2.112774     1.480439  0.523477  
14  2.023643     1.709564  0.824577  
17  3.236947     1.763567  -0.03942  
19  3.214152      2.10949 -0.244947  
20   2.14044     1.520908 -0.235322  
26  2.219424     1.584569 -0.194056  
27  3.677466     1.412285  0.075575  
28  3.100112     2.234986  0.392555  
34  2.840524     1.939714   0.59669  
35  3.693367      1.91998  0.296334